# Methods Notebook: Extracting Call Samples for KMeans Clustering

## Imports Section:

In [1]:
import numpy as np
import pandas as pd
import random
import scipy
from scipy import stats
import datetime as dt
import dask.dataframe as dd

In [2]:
import librosa
import warnings

In [3]:
import glob
import matplotlib.pyplot as plt
from matplotlib import colors
import soundfile as sf
import matplotlib.patches as patches
from pathlib import Path

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

In [5]:
import sys

# append the path of the
# parent directory
sys.path.append('..')
sys.path.append('../src/')
sys.path.append('../src/models/bat_call_detector/batdetect2/')

import src.batdt2_pipeline as batdetect2_pipeline
import src.file_dealer as file_dealer
from bat_detect.detector import models
from cfg import get_config
from pipeline import pipeline
from utils.utils import gen_empty_df, convert_df_ravenpro
from bat_detect.utils import wavfile
import bat_detect.utils.detector_utils as du
import bat_detect.detector.compute_features as feats
import bat_detect.detector.post_process as pp

## Function Definitions (most important functions are in `src/`)
### Below are the functions used for plotting and visualization

In [6]:
def plot_audio_seg(audio_features, spec_features):
    audio_seg = audio_features['audio_seg']
    fs = audio_features['sample_rate']
    start = audio_features['start']
    duration = audio_features['duration']

    vmax = spec_features['vmax']
    vmin = spec_features['vmin']
    cmap = spec_features['cmap']
    nfft = spec_features['NFFT']

    plt.figure(figsize=(15, 5))
    plt.rcParams.update({'font.size': 24})
    plt.title(spec_features['plot_title'], fontsize=24)
    plt.specgram(audio_seg, NFFT=nfft, cmap=cmap, vmin=vmin, vmax=vmax)
    plt.yticks(ticks=np.linspace(0, 1, 6), labels=np.linspace(0, fs/2000, 6).astype('int'))
    plot_xtype = 'float'
    if (duration > 60):
        plot_xtype = 'int'
    plt.xticks(ticks=np.linspace(0, duration*(fs/2), 11), 
               labels=np.round(np.linspace(start, start+duration, 11, dtype=plot_xtype), 2), rotation=30)
    plt.ylabel("Frequency (kHz)")
    plt.xlabel("Time (s)")
    plt.show()


def plot_dets_over_audio(audio_features, spec_features, plot_dets):
    audio_seg = audio_features['audio_seg']
    fs = audio_features['sample_rate']
    start = audio_features['start']
    duration = audio_features['duration']

    vmax = spec_features['vmax']
    vmin = spec_features['vmin']
    cmap = spec_features['cmap']
    nfft = spec_features['NFFT']

    plt.figure(figsize=(15, 5))
    plt.rcParams.update({'font.size': 24})
    plt.title(spec_features['plot_title'], fontsize=24)
    plt.specgram(audio_seg, NFFT=nfft, cmap=cmap, vmin=vmin, vmax=vmax)

    ax = plt.gca()
    for i, row in plot_dets.iterrows():
        rect = patches.Rectangle(((row['start_time'] - start)*(fs/2), row['low_freq']/(fs/2)), 
                        (row['end_time'] - row['start_time'])*(fs/2), (row['high_freq'] - row['low_freq'])/(fs/2), 
                        linewidth=1, edgecolor='yellow', facecolor='none', alpha=0.8)
        ax.add_patch(rect)

    plt.yticks(ticks=np.linspace(0, 1, 6), labels=np.linspace(0, fs/2000, 6).astype('int'))
    plt.ylabel("Frequency (kHz)")
    plot_xtype = 'float'
    if (duration > 60):
        plot_xtype = 'int'
    plt.xticks(ticks=np.linspace(0, duration*(fs/2), 11), 
               labels=np.round(np.linspace(start, start+duration, 11, dtype=plot_xtype), 2), rotation=30)
    plt.xlabel("Time (s)")
    plt.twinx(ax=ax)
    plt.yticks(ticks=np.linspace(0, 1, 6), labels=np.linspace(0, fs/2000, 6).astype('int'))
    plt.ylabel("Frequency (kHz)")

    plt.show()

In [7]:
def load_audio_file(audio_file, time_exp_fact, target_samp_rate, scale=False, max_duration=False):
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore', category=wavfile.WavFileWarning)
        #sampling_rate, audio_raw = wavfile.read(audio_file)
        audio_raw, sampling_rate = librosa.load(audio_file, sr=None)

    if len(audio_raw.shape) > 1:
        raise Exception('Currently does not handle stereo files')
    sampling_rate = sampling_rate * time_exp_fact

    # resample - need to do this after correcting for time expansion
    sampling_rate_old = sampling_rate
    sampling_rate = target_samp_rate
    audio_raw = librosa.resample(audio_raw, orig_sr=sampling_rate_old, target_sr=sampling_rate, res_type='polyphase')

    # clipping maximum duration
    if max_duration is not False:
        max_duration = np.minimum(int(sampling_rate*max_duration), audio_raw.shape[0])
        audio_raw = audio_raw[:max_duration]
        
    # convert to float32 and scale
    audio_raw = audio_raw.astype(np.float32)
    if scale:
        audio_raw = audio_raw - audio_raw.mean()
        audio_raw = audio_raw / (np.abs(audio_raw).max() + 10e-6)

    return sampling_rate, audio_raw

In [8]:
def convert_raventxtdf_to_bd2df(raventxt_df):
    bd2_human_df = raventxt_df[::2].copy()
    bd2_human_df = bd2_human_df.drop(columns=['Selection', 'View', 'Channel'])
    bd2_human_df.rename(columns={'Begin Time (s)':'start_time',
                                'End Time (s)':'end_time',
                                'Low Freq (Hz)':'low_freq',
                                'High Freq (Hz)':'high_freq'}, inplace=True)
    bd2_human_df.sort_values('start_time', inplace=True)
    
    return bd2_human_df

def return_confusion_matrix_from_comparing_two_detectors(human_df, machine_df):
    false_positives = 0 ## machine made detection that human did not make
    true_positives = 0  ## machine made detection that human made
    true_negatives = 0  ## machine did not make detection that human also did not make
    false_negatives = 0 ## machine did not make detection that human made
    human_ind = 0
    machine_ind = 0

    while (human_ind < len(human_df))&(machine_ind < len(machine_df)):
        # print(human_ind, machine_ind)
        human_det = human_df.iloc[human_ind]

        machine_det = machine_df.iloc[machine_ind]

        machine_det_start_in_human_det_bounds = ((machine_det['start_time']>=human_det['start_time'])&(machine_det['start_time']<=human_det['end_time']))
        machine_det_end_in_human_det_bounds = ((machine_det['end_time']>=human_det['start_time'])&(machine_det['end_time']<=human_det['end_time']))
        human_det_start_in_machine_det_bounds = ((human_det['start_time']>=machine_det['start_time'])&(human_det['start_time']<=machine_det['end_time']))
        human_det_end_in_machine_det_bounds = ((human_det['end_time']>=machine_det['start_time'])&(human_det['end_time']<=machine_det['end_time']))

        machine_det_in_human_bounds = machine_det_start_in_human_det_bounds|machine_det_end_in_human_det_bounds
        human_det_in_machine_bounds = human_det_start_in_machine_det_bounds|human_det_end_in_machine_det_bounds

        # if human_ind>=295:
        #     print('bug')
        #     print(human_det['start_time'], human_det['end_time'])
        #     print(machine_det['start_time'], machine_det['end_time'])
        if (machine_det_in_human_bounds|human_det_in_machine_bounds):
            true_positives += 1
            machine_ind+=1
            human_ind+=1
        else:
            if (machine_det['end_time']<human_det['start_time']):
                false_positives += 1
                machine_ind+=1
            elif (machine_det['start_time']>human_det['end_time']):
                false_negatives += 1
                human_ind+=1
            else:
                true_negatives += 1

    if (machine_ind<len(machine_df)):
        false_positives += (len(machine_df) - (machine_ind))
    if (human_ind<len(human_df)):
        false_negatives += (len(human_df) - (human_ind))

    assert((true_positives+false_negatives)==len(human_df))
    assert((false_positives+true_positives)==len(machine_df))

    return {'true_positives':true_positives, 'false_positives':false_positives, 'false_negatives':false_negatives, 'true_negatives':true_negatives}

In [9]:
def run_detector_on_audio_data(model, args, params, audio_full, plot_file, sampling_rate):
    # store temporary results here
    predictions = []
    spec_feats  = []
    cnn_feats   = []
    spec_slices = []

    # get time expansion  factor
    time_exp = args['time_expansion_factor']

    params['detection_threshold'] = args['detection_threshold']
    top_n=5
    # clipping maximum duration
    duration_full = audio_full.shape[0] / float(sampling_rate)

    return_np_spec = args['spec_features'] or args['spec_slices']

    # loop through larger file and split into chunks
    # TODO fix so that it overlaps correctly and takes care of duplicate detections at borders
    num_chunks = int(np.ceil(duration_full/args['chunk_size']))
    for chunk_id in tqdm(range(num_chunks)):
        # chunk
        chunk_time   = args['chunk_size']*chunk_id
        chunk_length = int(sampling_rate*args['chunk_size'])
        start_sample = chunk_id*chunk_length
        end_sample   = np.minimum((chunk_id+1)*chunk_length, audio_full.shape[0])
        audio = audio_full[start_sample:end_sample]

        # load audio file and compute spectrogram
        duration, spec, spec_np = du.compute_spectrogram(audio, sampling_rate, params, return_np_spec)

        # evaluate model
        with torch.no_grad():
            outputs = model(spec, return_feats=args['cnn_features'])

        # run non-max suppression
        pred_nms, features = pp.run_nms(outputs, params, np.array([float(sampling_rate)]))
        pred_nms = pred_nms[0]
        pred_nms['start_times'] += chunk_time
        pred_nms['end_times'] += chunk_time

        # if we have a background class
        if pred_nms['class_probs'].shape[0] > len(params['class_names']):
            pred_nms['class_probs'] = pred_nms['class_probs'][:-1, :]

        predictions.append(pred_nms)

        # extract features - if there are any calls detected
        if (pred_nms['det_probs'].shape[0] > 0):
            if args['spec_features']:
                spec_feats.append(feats.get_feats(spec_np, pred_nms, params))

            if args['cnn_features']:
                cnn_feats.append(features[0])

            if args['spec_slices']:
                spec_slices.extend(feats.extract_spec_slices(spec_np, pred_nms, params))

    # convert the predictions into output dictionary
    file_id = plot_file.name
    predictions, spec_feats, cnn_feats, spec_slices =\
                du.merge_results(predictions, spec_feats, cnn_feats, spec_slices)
    results = du.convert_results(file_id, time_exp, duration_full, params,
                                predictions, spec_feats, cnn_feats, spec_slices)

    # summarize results
    if not args['quiet']:
        num_detections = len(results['pred_dict']['annotation'])
        print('{}'.format(num_detections) + ' call(s) detected above the threshold.')

    # print results for top n classes
    if not args['quiet'] and (num_detections > 0):
        class_overall = pp.overall_class_pred(predictions['det_probs'], predictions['class_probs'])
        print('species name'.ljust(30) + 'probablity present')
        for cc in np.argsort(class_overall)[::-1][:top_n]:
            print(params['class_names'][cc].ljust(30) + str(round(class_overall[cc], 3)))

    annotations = results['pred_dict']['annotation']
    out_df = gen_empty_df()
    if annotations:
        out_df = pd.DataFrame.from_records(annotations) 

    return out_df

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net_params = torch.load('../src/models/bat_call_detector/batdetect2/models/Net2DFast_UK_same.pth.tar', map_location=device)
params = net_params['params']
params['device'] = device
model = models.Net2DFast(params['num_filters'], num_classes=len(params['class_names']),
                            emb_dim=params['emb_dim'], ip_height=params['ip_height'],
                            resize_factor=params['resize_factor'])
model.load_state_dict(net_params['state_dict'])
model = model.to(params['device'])
model.eval()

Net2DFast(
  (conv_dn_0): ConvBlockDownCoordF(
    (conv): Conv2d(2, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_dn_1): ConvBlockDownCoordF(
    (conv): Conv2d(33, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_dn_2): ConvBlockDownCoordF(
    (conv): Conv2d(65, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv_bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_dn_3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_dn_3_bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_1d): Conv2d(256, 256, kernel_size=(16, 1), stride=(1, 1))
  (conv_1d_bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (a

In [11]:
# batdetect2_df = pd.read_csv(f'../../duty-cycle-investigation/data/raw/{site}/bd2__{site}_{wav_filename}.csv', sep=',')
# batdetect2_df

In [12]:
# # Below we filter our detections dataframe to only get the detections corresponding to the above audio file
# audio_file = sf.SoundFile(plot_file)
# fs = audio_file.samplerate
# start = 210
# duration = 120
# audio_file.seek(int(fs*start))
# audio_seg = audio_file.read(int(fs*duration))
# lower_dB = -40
# upper_dB = 0
# vmin = 20*np.log10(np.max(audio_seg)) + lower_dB # hide anything below lower_dB dB
# vmax = 20*np.log10(np.max(audio_seg)) + upper_dB # hide anything above upper_dB dB
# audio_features = dict()
# audio_features['file_path'] = Path(plot_file)
# audio_features['audio_seg'] = audio_seg
# audio_features['sample_rate'] = fs
# audio_features['start'] = start
# audio_features['duration'] = duration

# spec_features = dict()
# spec_features['NFFT'] = 1024 # When segments are short, NFFT should also be small to best see calls (must always be > 128)
# spec_features['cmap'] = 'jet' # This colormap shows best contrast between noise and signals
# spec_features['vmin'] = vmin
# spec_features['vmax'] = vmax

In [13]:
file_sites = {'20220730_053000':'Carp',
 '20220727_083000':'Foliage',
 '20220829_090000':'Foliage',
 '20220826_070000':'Central'}

file_keys = list(file_sites.keys())
file_keys

['20220730_053000', '20220727_083000', '20220829_090000', '20220826_070000']

In [14]:
save_dir = Path(f'bd2__det_thresh_sweep_20220730_053000')
save_loc = Path(f"bd2__0p1_chunksize3_20220730_053000.csv")

In [15]:
save_dir / save_loc

PosixPath('bd2__det_thresh_sweep_20220730_053000/bd2__0p1_chunksize3_20220730_053000.csv')

In [39]:
# resamp_audio_seg = audio_data_resampled[int(resampling_rate*start):int(resampling_rate*(start+duration))]

for file_key in file_keys:
    wav_filename = file_key
    site = file_sites[file_key]
    print(wav_filename, site)
    plot_file = Path(f'../../Documents/{wav_filename}/input/{wav_filename}.WAV')
    resampling_rate, audio_data_resampled = load_audio_file(plot_file, 1, 256000)

    raventxt_human_df = pd.read_csv(f'../../Documents/Research/mila_files/mila-human-wav-txt/{wav_filename}.txt', sep='\t')
    bd2_human_df = convert_raventxtdf_to_bd2df(raventxt_human_df)
    bd2_human_df.reset_index(inplace=True, drop=True)

    for thresh in np.arange(0.1, 0.7, 0.02):
        args = dict()
        args['time_expansion_factor'] = 1.0
        args['detection_threshold'] = round(thresh,2)
        args['chunk_size'] = 3
        args['spec_slices'] = False
        args['quiet'] = False
        args['spec_features'] = False
        args['cnn_features'] = True
        threshold_tag = f"threshold{str(args['detection_threshold']).replace('.','p')}"
        print(args['detection_threshold'], str(args['detection_threshold']), threshold_tag)
        save_dir = Path(f'bd2__det_thresh_sweep_{wav_filename}')
        save_loc = Path(f"bd2__{threshold_tag}_chunksize{args['chunk_size']}_{plot_file.name.split('.')[0]}.csv")
        print("saving to", (save_dir/save_loc))
        if not(save_dir.exists()):
            save_dir.mkdir()
        if not((save_dir/save_loc).exists()):
            batdetect2_df_mod = run_detector_on_audio_data(model, args, params, audio_data_resampled, plot_file, resampling_rate)
            batdetect2_df_mod.to_csv((save_dir/save_loc))
        else:
            batdetect2_df_mod = pd.read_csv((save_dir/save_loc), sep=',')
        print(return_confusion_matrix_from_comparing_two_detectors(bd2_human_df, batdetect2_df_mod))

20220730_053000 Carp
0.1 0.1 threshold0p1
saving to bd2__det_thresh_sweep_20220730_053000/bd2__threshold0p1_chunksize3_20220730_053000.csv
{'true_positives': 665, 'false_positives': 296, 'false_negatives': 7, 'true_negatives': 0}
0.12 0.12 threshold0p12
saving to bd2__det_thresh_sweep_20220730_053000/bd2__threshold0p12_chunksize3_20220730_053000.csv
{'true_positives': 661, 'false_positives': 219, 'false_negatives': 11, 'true_negatives': 0}
0.14 0.14 threshold0p14
saving to bd2__det_thresh_sweep_20220730_053000/bd2__threshold0p14_chunksize3_20220730_053000.csv
{'true_positives': 659, 'false_positives': 179, 'false_negatives': 13, 'true_negatives': 0}
0.16 0.16 threshold0p16
saving to bd2__det_thresh_sweep_20220730_053000/bd2__threshold0p16_chunksize3_20220730_053000.csv
{'true_positives': 655, 'false_positives': 153, 'false_negatives': 17, 'true_negatives': 0}
0.18 0.18 threshold0p18
saving to bd2__det_thresh_sweep_20220730_053000/bd2__threshold0p18_chunksize3_20220730_053000.csv
{'true

100%|██████████| 599/599 [07:59<00:00,  1.25it/s]


474 call(s) detected above the threshold.
species name                  probablity present
Pipistrellus nathusii         0.678
Nyctalus leisleri             0.131
Pipistrellus pipistrellus     0.038
Eptesicus serotinus           0.035
Myotis daubentonii            0.026
{'true_positives': 456, 'false_positives': 18, 'false_negatives': 216, 'true_negatives': 0}
0.46 0.46 threshold0p46
saving to bd2__det_thresh_sweep_20220730_053000/bd2__threshold0p46_chunksize3_20220730_053000.csv


100%|██████████| 599/599 [08:36<00:00,  1.16it/s]


428 call(s) detected above the threshold.
species name                  probablity present
Pipistrellus nathusii         0.683
Nyctalus leisleri             0.133
Eptesicus serotinus           0.036
Pipistrellus pipistrellus     0.033
Myotis daubentonii            0.025
{'true_positives': 415, 'false_positives': 13, 'false_negatives': 257, 'true_negatives': 0}
0.48 0.48 threshold0p48
saving to bd2__det_thresh_sweep_20220730_053000/bd2__threshold0p48_chunksize3_20220730_053000.csv


...

245 call(s) detected above the threshold.
species name                  probablity present
Pipistrellus nathusii         0.557
Pipistrellus pipistrellus     0.205
Nyctalus leisleri             0.139
Eptesicus serotinus           0.026
Rhinolophus ferrumequinum     0.018
{'true_positives': 245, 'false_positives': 0, 'false_negatives': 2412, 'true_negatives': 0}
0.66 0.66 threshold0p66
saving to bd2__det_thresh_sweep_20220826_070000/bd2__threshold0p66_chunksize3_20220826_070000.csv


100%|██████████| 599/599 [05:46<00:00,  1.73it/s]


160 call(s) detected above the threshold.
species name                  probablity present
Pipistrellus nathusii         0.542
Pipistrellus pipistrellus     0.212
Nyctalus leisleri             0.151
Eptesicus serotinus           0.025
Rhinolophus ferrumequinum     0.018
{'true_positives': 160, 'false_positives': 0, 'false_negatives': 2497, 'true_negatives': 0}
0.68 0.68 threshold0p68
saving to bd2__det_thresh_sweep_20220826_070000/bd2__threshold0p68_chunksize3_20220826_070000.csv


100%|██████████| 599/599 [06:04<00:00,  1.64it/s]


85 call(s) detected above the threshold.
species name                  probablity present
Pipistrellus nathusii         0.53
Pipistrellus pipistrellus     0.232
Nyctalus leisleri             0.152
Eptesicus serotinus           0.019
Rhinolophus ferrumequinum     0.018
{'true_positives': 85, 'false_positives': 0, 'false_negatives': 2572, 'true_negatives': 0}


In [38]:
wav_filename = file_keys[3]
site = file_sites[wav_filename]
print(wav_filename, site)
plot_file = Path(f'../../Documents/{wav_filename}/input/{wav_filename}.WAV')

raventxt_human_df = pd.read_csv(f'../../Documents/Research/mila_files/mila-human-wav-txt/{wav_filename}.txt', sep='\t')
bd2_human_df = convert_raventxtdf_to_bd2df(raventxt_human_df)
bd2_human_df.reset_index(inplace=True, drop=True)

# Below we filter our detections dataframe to only get the detections corresponding to the above audio file
audio_file = sf.SoundFile(plot_file)
fs = audio_file.samplerate
start = 900
duration = 600
audio_file.seek(int(fs*start))
audio_seg = audio_file.read(int(fs*duration))
lower_dB = -60
upper_dB = 0
vmin = 20*np.log10(np.max(audio_seg)) + lower_dB # hide anything below lower_dB dB
vmax = 20*np.log10(np.max(audio_seg)) + upper_dB # hide anything above upper_dB dB
audio_features = dict()
audio_features['file_path'] = Path(plot_file)
audio_features['audio_seg'] = audio_seg
audio_features['sample_rate'] = fs
audio_features['start'] = start
audio_features['duration'] = duration

spec_features = dict()
spec_features['NFFT'] = 1024 # When segments are short, NFFT should also be small to best see calls (must always be > 128)
spec_features['cmap'] = 'jet' # This colormap shows best contrast between noise and signals
spec_features['vmin'] = vmin
spec_features['vmax'] = vmax

spec_features['plot_title'] = f"Mila's detections on {audio_features['file_path'].name} in {site}"
# plot_dets_over_audio(audio_features, spec_features, bd2_human_df)

for det_thresh in np.round(np.arange(0.2, 0.6, 0.1), 2):
    threshold_tag = f"threshold{str(det_thresh).replace('.','p')}"
    save_dir = Path(f'bd2__det_thresh_sweep_{wav_filename}')
    save_loc = Path(f"bd2__{threshold_tag}_chunksize{args['chunk_size']}_{plot_file.name.split('.')[0]}.csv")
    batdetect2_df_mod = pd.read_csv((save_dir/save_loc), sep=',')
    
    spec_features['plot_title'] = f"Batdetect2 detections on {audio_features['file_path'].name} in {site} Threshold: {det_thresh}"
    # plot_dets_over_audio(audio_features, spec_features, batdetect2_df_mod)
    print(return_confusion_matrix_from_comparing_two_detectors(bd2_human_df, batdetect2_df_mod))

20220826_070000 Central
{'true_positives': 2593, 'false_positives': 1326, 'false_negatives': 64, 'true_negatives': 0}
{'true_positives': 2422, 'false_positives': 806, 'false_negatives': 235, 'true_negatives': 0}
{'true_positives': 2092, 'false_positives': 336, 'false_negatives': 565, 'true_negatives': 0}
{'true_positives': 1542, 'false_positives': 34, 'false_negatives': 1115, 'true_negatives': 0}


In [18]:
# for det_thresh in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
#     batdetect2_df = pd.read_csv(f"bd2__threshold{str(det_thresh).replace('.','p')}_chunksize3_{plot_file.name.split('.')[0]}.csv", sep=',')
#     print(return_confusion_matrix_from_comparing_two_detectors(bd2_human_df, batdetect2_df))